In [5]:
import numpy as np
from wrf import getvar, interplevel, to_np, latlon_coords
import netCDF4 as nc

def change(var,nc_file):
    vari=np.empty_like(nc_file.variables[var]);
    for hr in range(time):
        for lvl in range(levels):
            vari[hr][lvl]=perturbar(nc_file.variables[var][hr][lvl], per);
    return vari;

def perturbar(var, pert):
    shap= var.shape
    var_pert = var + pert * np.random.randn(shap[0],shap[1]);
    return var_pert.astype(dtype='float32');

def create_file_from_source(src, trg_file):
    trg = nc.Dataset(trg_file, mode='w')

    # Create the dimensions of the file
    for name, dim in src.dimensions.items():
        trg.createDimension(name, len(dim) if not dim.isunlimited() else None)

    # Copy the global attributes
    trg.setncatts({a:src.getncattr(a) for a in src.ncattrs()})

    # Create the variables in the file
    for name, var in src.variables.items():
        trg.createVariable(name, var.dtype, var.dimensions)

        # Copy the variable attributes
        trg.variables[name].setncatts({a:var.getncattr(a) for a in var.ncattrs()})

        # Copy the variables values (as 'f4' eventually)
        if name=='U' or name=='V' or name=='W' or name=='P' or name=='T':
            trg.variables[name][:] = change(name,src)
        else:
            trg.variables[name][:] = src.variables[name][:]

    # Save the file
    trg.close()

In [7]:
input_file ='/home/syseng/omejiaa/workspace/wrfout.nc'
out_file ='/home/syseng/omejiaa/workspace/wrfout_perturbado.nc'
per        = 0.01
nc_file    = nc.Dataset(input_file)
date       = getvar(nc_file,'Times')
time       = nc_file.variables['ZNU'].shape[0]
levels     = nc_file.variables['ZNU'].shape[1]
create_file_from_source(nc_file, out_file)